In [270]:
import random
from enum import Enum
from bitstring import BitArray

In [271]:
def bitfield_status(bitfield_payload, metadata) -> str:
    # Assuming that the bitfield header contains bitfield, aka: message id == 5:
    
    if bitfield_payload:
        if len(BitArray(bitfield_payload).bin) == metadata['bitfield_length']:
            if BitArray(bitfield_payload).bin.count('1') == metadata['bitfield_length'] - metadata['bitfield_spare']:
                return 'seeder' # Peer has 100% of data 
            else:
                return 'leecher' # Peer has x% of data 
    
    return 'stale' 

In [272]:
# Test function
def test_bitfield_status(debug=True):
    """ OVERKILL x99
    Currently overkill, but should be ok when implementing bitfield status for every peer 
    (assuming that we can get bitfield status of different types)
    """

    # Correct response 
    # Sate 0 == seeder
    # Sate 1 == leecher
    # Sate 2 == unknown

    for number_of_pieces in [9,21,123,4324,121]: # BUG: now is under 9 in loop
        for state in [0,1,2]:

            if state == 0: bitfield_list =  [1] * number_of_pieces
            if state == 1: bitfield_list =  [0] * (number_of_pieces-1) + [1]
            if state == 2: bitfield_list =  b'\x00'
            if debug: print(bitfield_list)

            
            bitfield_bin_padded = ''.join(map(str, bitfield_list)).ljust((len(bitfield_list) + 7) // 8 * 8, '0')
            if debug: print("f", bitfield_bin_padded)

            bitfield_bytes = int(bitfield_bin_padded, 2).to_bytes((len(bitfield_bin_padded) + 7) // 8, byteorder='big')
            if debug: print(bitfield_bytes, state)

            metadata = {}
            metadata['bitfield_length'] = ((number_of_pieces + 7) // 8) * 8
            metadata['bitfield_spare'] = metadata['bitfield_length'] - number_of_pieces

            # Test all states
            if state == 0: assert bitfield_status(bitfield_bytes, metadata) == "seeder"
            if state == 1: assert bitfield_status(bitfield_bytes, metadata) == "leecher"
            if state == 2: assert bitfield_status(bitfield_bytes, metadata) == "unknown"


        # Bitfield should always be bytes
        assert bitfield_status(b'', None) == "unknown"


In [273]:
test_bitfield_status()

[1, 1, 1, 1, 1, 1, 1, 1, 1]
f 1111111110000000
b'\xff\x80' 0
[0, 0, 0, 0, 0, 0, 0, 0, 1]
f 0000000010000000
b'\x00\x80' 1
b'\x00'
f 00000000
b'\x00' 2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
f 111111111111111111111000
b'\xff\xff\xf8' 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
f 000000000000000000001000
b'\x00\x00\x08' 1
b'\x00'
f 00000000
b'\x00' 2
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
f 11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111100000
b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xe0' 0
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 